# Verify AirFlow is ready to Start DAG

In [1]:
import boto3
import requests

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

dag_name = 'bert_reviews'

In [2]:
%store -r airflow_bucket_name
%store -r airflow_env_name
%store -r team_role_arn

In [3]:
mwaa = boto3.client('mwaa')
mwaa_status = ""

def get_airflow_check():
    response = mwaa.get_environment(
        Name=airflow_env_name
    )
    mwaa_status = response["Environment"]["Status"]
    return mwaa_status


mwaa_status = get_airflow_check()
if(mwaa_status != 'AVAILABLE'):
    print('[ERROR] Cannot find MWAA {}.'.format(airflow_env_name))
else:    
    print('Sucess! {} is ready!'.format(airflow_env_name))

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the GetEnvironment operation: Environment mwaa-us-east-1-835319576252 not found.

# Triggering DAG from MWAA

In [4]:
mwaa_cli_token = mwaa.create_cli_token(
    Name=airflow_env_name
)

cli_token = 'Bearer ' + mwaa_cli_token['CliToken']
mwaa_web_server_hostname = 'https://' + mwaa_cli_token['WebServerHostname'] + '/aws_mwaa/cli'

raw_data = 'trigger_dag {}'.format(dag_name)

response = requests.post(
        mwaa_web_server_hostname,
        headers={
            'Authorization': cli_token,
            'Content-Type': 'text/plain'
            },
        data=raw_data
        )

if (response.status_code != 200):
    print('ERROR: DAG: {} failed to get triggered!'.format(dag_name))
else:
    print('Sucess! DAG: {} was triggered successfuly'.format(dag_name))

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the CreateCliToken operation: Environment not found

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}